In [75]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import auc
from catboost import CatBoostClassifier

In [2]:
data = pd.read_csv('train.csv')
data = data[list(data.columns[:1]) + list(data.columns[2:]) + list(data.columns[1:2])]
data.head()

,id,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,...,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216,label
0,1,7,0,3,0,2,3,0,6,0,...,4,2,2,0,13,0,11,1,3,1
1,2,0,11,0,0,10,1,0,0,4,...,2,0,0,2,8,1,13,0,4,1
2,3,9,0,3,0,1,3,0,4,0,...,11,2,0,0,4,0,2,0,0,0
3,4,0,9,3,2,25,0,4,0,0,...,14,1,0,0,0,3,0,17,1,0
4,5,0,0,0,0,2,5,0,0,0,...,12,0,3,0,4,0,24,4,0,0


In [3]:
X_train = data.iloc[:,:-1]
y_train = data.iloc[:,-1:]

In [4]:
clf = CatBoostClassifier(random_seed=42)
clf = clf.fit(X_train, y_train, verbose=False)

In [5]:
best_features = np.where(np.array(clf.feature_importances_) >= 1)[0]

In [6]:
X_train = X_train.iloc[:,best_features]

In [17]:
ones = np.where(np.array(y_train['label']) == 1)[0]
zeros = np.where(np.array(y_train['label']) == 0)[0]

In [65]:
pairs = []
for i in range(10):
    picked_ones = np.random.choice(len(ones), min(len(ones), len(zeros)))
    picked_zeros = np.random.choice(len(zeros), min(len(ones), len(zeros)))
    picked_ones = np.append(picked_ones, picked_zeros)
    pairs.append(picked_ones)
pairs = np.array(pairs)

In [91]:
X_test = pd.read_csv('test.csv')
X_test = X_test.iloc[:,best_features]
X_test.head()

,feat8,feat10,feat19,feat44,feat61,feat68,feat75,feat80,feat100,feat101,...,feat161,feat168,feat170,feat171,feat179,feat181,feat183,feat184,feat192,feat216
0,2,0,5,3,5,2,1,0,2,6,...,7,7,3,10,7,1,7,1,4,3
1,9,3,4,8,10,8,3,9,6,2,...,7,1,9,1,9,2,8,8,6,0
2,1,9,2,2,0,0,0,0,1,47,...,4,5,0,57,1,0,0,0,0,1
3,7,9,13,7,6,2,1,6,9,2,...,16,2,2,1,3,0,1,9,5,3
4,13,3,12,11,3,1,7,9,3,2,...,23,0,8,0,2,8,11,8,14,7


In [92]:
predictions = []
for i in range(10):
    logreg = LogisticRegressionCV(cv=5, solver='liblinear', n_jobs=-1, random_state=42)
    logreg.fit(X_train.iloc[pairs[i],], np.array(y_train.iloc[pairs[i],]).ravel())
    predictions.append(logreg.predict(X_test))

In [104]:
prediction = []
for i in np.sum(predictions, axis=0):
    if i >= 5:
        prediction.append(1)
    else:
        prediction.append(0)

In [113]:
csv = pd.DataFrame()
csv["id"] = range(1, 9051)
csv["label"] = pd.DataFrame(prediction).astype(int)
csv.to_csv('logreg_18_13.csv', index=None)